### Load the COCO data

In [ ]:
!wget http://images.cocodataset.org/zips/train2017.zip # Example COCO download
!unzip train2017.zip # Unzip the downloaded file

### To find the number of images in the train2017 dataset

In [ ]:
import os

train_image_dir = '/content/train2017'  # Replace with your actual directory path

if os.path.exists(train_image_dir):
    image_files = [f for f in os.listdir(train_image_dir) if f.lower().endswith(('.png', '.jpg', '.jpeg'))] # only count image files
    num_images = len(image_files)
    print(f"Number of images in train2017: {num_images}")
else:
    print(f"Directory '{train_image_dir}' does not exist.")

### Download annotation file for trainval2017 data

In [ ]:
!wget http://images.cocodataset.org/annotations/annotations_trainval2017.zip
!unzip annotations_trainval2017.zip

### To see number of items in annotations file (instances_train2017)

In [ ]:
import json

annotation_file = '/content/annotations/annotations/instances_train2017.json'

try:
    with open(annotation_file, 'r') as f:
        data = json.load(f)

    num_images = len(data['images'])
    num_annotations = len(data['annotations'])

    print(f"Number of images in instances_train2017.json: {num_images}")
    print(f"Number of annotations in instances_train2017.json: {num_annotations}")

except FileNotFoundError:
    print(f"Error: File '{annotation_file}' not found.")
except json.JSONDecodeError:
    print(f"Error: File '{annotation_file}' is not a valid JSON file.")
except KeyError as e:
    print(f"Error: Key '{e}' not found in the JSON file.")
except Exception as e:
    print(f"An unexpected error occurred: {e}")

### Load a few elements from the instances_train2017 file to see the structure in which data is stored

In [ ]:
import json

annotation_file = '/content/annotations/annotations/instances_train2017.json'

try:
    with open(annotation_file, 'r') as f:
        data = json.load(f)

    # Print the first few image entries
    print("First few image entries:")
    for image in data['images'][10:20]:  # Print the first 5 image entries
        print(image)

    print("\nFirst few annotation entries:")
    for annotation in data['annotations'][10:20]: # print the first 5 annotation entries
        print(annotation)

except FileNotFoundError:
    print(f"Error: File '{annotation_file}' not found.")
except json.JSONDecodeError:
    print(f"Error: File '{annotation_file}' is not a valid JSON file.")
except KeyError as e:
    print(f"Error: Key '{e}' not found in the JSON file.")
except Exception as e:
    print(f"An unexpected error occurred: {e}")

## create subset of 8k images


In [ ]:
import os
import random
import shutil
import json

train_image_dir = '/content/train2017'
train_subset_dir = '/content/train_subset'
annotation_file = '/content/annotations/annotations/instances_train2017.json'
num_images_to_select = 8000

# Create the subset directory
os.makedirs(train_subset_dir, exist_ok=True)

# Get a list of image files
image_files = [f for f in os.listdir(train_image_dir) if f.lower().endswith(('.png', '.jpg', '.jpeg'))]

# Randomly select 8000 images
selected_images = random.sample(image_files, min(num_images_to_select, len(image_files)))

# Set the random seed
random_seed = 42 
random.seed(random_seed)

# Copy the selected images to the subset directory
for image_file in selected_images:
    src_path = os.path.join(train_image_dir, image_file)
    dst_path = os.path.join(train_subset_dir, image_file)
    shutil.copy(src_path, dst_path)

print(f"Copied {len(selected_images)} images to {train_subset_dir}")

# Process the annotation file.
with open(annotation_file, 'r') as f:
    annotations = json.load(f)

image_id_to_filename = {img['id']: img['file_name'] for img in annotations['images']}
filename_to_image_id = {v: k for k, v in image_id_to_filename.items()}

subset_image_ids = [filename_to_image_id[f] for f in selected_images]

# Filter the annotations.
subset_images = [img for img in annotations['images'] if img['id'] in subset_image_ids]
subset_annotations = [ann for ann in annotations['annotations'] if ann['image_id'] in subset_image_ids]

# Create a new annotation file.
subset_annotation_data = {'images': subset_images, 'annotations': subset_annotations}

with open('/content/subset_train_annotations.json', 'w') as f:
    json.dump(subset_annotation_data, f)

print("Created subset train annotation file.")

In [ ]:
!zip -r train_subset.zip /content/train_subset #to zip the subset directory
!zip -r subset_train_annotations.zip /content/subset_train_annotations.json #to zip the annotation file

#### to see what does each category_id correspond to

In [ ]:
import json

annotation_file = '/content/annotations/annotations/instances_train2017.json'  # Replace with your actual path

try:
    with open(annotation_file, 'r') as f:
        data = json.load(f)

    categories = data.get('categories', [])

    if not categories:
        print("No category information found in the annotation file.")
    else:
        print("Category Mapping:")
        for category in categories:
            print(f"  ID: {category['id']}, Name: {category['name']}")

except FileNotFoundError:
    print(f"Error: File '{annotation_file}' not found.")
except json.JSONDecodeError:
    print(f"Error: File '{annotation_file}' is not a valid JSON file.")
except KeyError as e:
    print(f"Error: Key '{e}' not found in the JSON file.")
except Exception as e:
    print(f"An unexpected error occurred: {e}")

#### ## Creating mask: storing category_id along with image name to clearly see which category_id it consists of

In [ ]:
import os
import json
import numpy as np
from PIL import Image, ImageDraw
from tqdm import tqdm
import matplotlib.pyplot as plt

def create_segmentation_masks(image_dir, annotation_file, output_dir):
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    with open(annotation_file, 'r') as f:
        annotations = json.load(f)

    image_annotations = {}
    for ann in annotations['annotations']:
        image_id = ann['image_id']
        if image_id not in image_annotations:
            image_annotations[image_id] = []
        image_annotations[image_id].append(ann)

    image_id_to_filename = {img['id']: img['file_name'] for img in annotations['images']}

    for image_id, anns in tqdm(image_annotations.items(), desc="Processing Images"):
        filename = image_id_to_filename[image_id]
        image_path = os.path.join(image_dir, filename)

        try:
            image = Image.open(image_path).convert('RGB')
            width, height = image.size

            mask = np.zeros((height, width), dtype=np.uint8)
            mask_image = Image.new('L', (width, height), 0)
            draw = ImageDraw.Draw(mask_image)

            present_category_ids = sorted(list(set()))  # Sorted list for consistent naming

            for ann in anns:
                segmentation = ann['segmentation']
                category_id = ann['category_id']
                present_category_ids.append(category_id)  # Add category_id to the list

                if isinstance(segmentation, list):  # Polygon segmentation
                    for polygon in segmentation:
                        polygon_points = [(polygon[i], polygon[i + 1]) for i in range(0, len(polygon), 2)]
                        draw.polygon(polygon_points, fill=int(category_id * 100))
                        mask = np.maximum(mask, np.array(mask_image))
                        mask_image = Image.new('L', (width, height), 0)

            mask_image = Image.fromarray(mask)

            # Create the category ID string for the filename
            category_id_str = '_'.join(map(str, sorted(list(set(present_category_ids)))))  # Unique sorted IDs

            mask_filename = f"{os.path.splitext(filename)[0]}_mask_cats_{category_id_str}.png"
            mask_path = os.path.join(output_dir, mask_filename)
            mask_image.save(mask_path)

            print(f"Mask saved to {mask_path}")

        except FileNotFoundError:
            print(f"Error: Image not found: {image_path}")
        except Exception as e:
            print(f"Error processing image {image_path}: {e}")

# Example usage (replace with your actual paths):
image_dir = '/content/train_subset'
annotation_file = '/content/subset_train_annotations.json'
output_dir = '/content/masks_2'

create_segmentation_masks(image_dir, annotation_file, output_dir)

### ## Visualizing specific images from train_subset

In [ ]:
from PIL import Image
import matplotlib.pyplot as plt
import os

image_dir = '/content/train_subset'  # Replace with the actual path to your train_subset directory
image_filename = '000000000643.jpg'  # Replace with the actual filename of the image you want to see

image_path = os.path.join(image_dir, image_filename)

try:
    image = Image.open(image_path)
    plt.imshow(image)
    plt.title(image_filename)
    plt.show()
except FileNotFoundError:
    print(f"Error: Image file '{image_path}' not found.")
except Exception as e:
    print(f"An unexpected error occurred: {e}")

In [ ]:
!zip -r masks_2.zip /content/masks_2 #to zip the masks directory